# Limbus Assignment

Assignment Submitted on 23-October-2020

Author: Sridatta Kolli

#importing all the necessary modules
import numpy as np 
import os
import copy
from math import *
import matplotlib.pyplot as plt
from functools import reduce
import pydicom
import pandas as pd
import scipy.spatial as ss


In [105]:
#function to read each file from the path given where it collects all RT_data and CT_images into list called RT and CT
def Load_Scan(path):
    slices = [pydicom.dcmread(path + '\\' + s, force=True) for s in               
              os.listdir(path)]
    print(len(slices))
    CT = []
    RT=[]
    for s in slices:
        if 'SliceLocation' not in s:
            RT.append(s)
        else:
            CT.append(s)
    return RT,CT

In [106]:
#Collecting individual scan CT_images count
def CT_Images_Count(images, ids):
    count = 0
    for i in range(len(images)):
        if ids == images[i].PatientID:
            count= count+1
    return count         
        

In [107]:
#calculating the volume of heart using ScipySpatial module
def Volume(arr_points):
    points = arr_points
    hull = ss.ConvexHull(points)
    return(hull.volume/1000)

In [108]:
#collecting all the points from Contour Data from Index where ROIName is HEART
def CollectingPoints(seq, index_no): 
    count = 1
    index_no = index_no-1
    arr_ls = seq.ROIContourSequence[index_no].ContourSequence[0].ContourData
    arr_ls = np.array(arr_ls)
    length = int(len(arr_ls)/3)
    arr_ls = arr_ls.reshape((length,3))

    for i in range(1,len(seq.ROIContourSequence[index_no].ContourSequence)):
        vec = seq.ROIContourSequence[index_no].ContourSequence[i].ContourData
        arr = np.array(vec)
        length = int(len(arr)/3)
        arr = arr.reshape((length,3))
        
        arr_ls= np.append(arr_ls,arr, axis =0)
        count = count+1
    volume = Volume(arr_ls)
    print('Volume:',volume,'Count of Contour Data',count)

In [109]:
# set path and load files 
path ='C:\\Users\\kolli\\Desktop\\Limbus\\dicom_files\\dicom_files'
RT_data,CT_images = Load_Scan(path)

442


In [110]:
del RT_data[0]
len(RT_data),len(CT_images)

(3, 438)

In [111]:
#generating the patient IDs from all files given
Patient_IDs=[]
patient_id = []
for i in range(len(CT_images)):
    patient_id.append(CT_images[i].PatientID) 
Patient_IDs =list(set(patient_id))
Patient_IDs

['ECI002000', '12778', 'ECI003000']

In [112]:
#function Calling for collecting each scan CT_images
Image_Count = []
for ls in patient_IDs:
    c = CT_Images_Count(CT_images, ls)
    Image_Count.append(c)

In [113]:
#CT_images count for each individual scan
Image_Count

[112, 220, 106]

In [114]:
#finally volume calculation at the required index and ROIName
for i in range(0,len(RT_data)):
    data = RT_data[i]
    print(i)
    for j in range(len(data.StructureSetROISequence)):
        if (data.StructureSetROISequence[j].ROIName == 'HEART'):
            num = seq.StructureSetROISequence[1].ROINumber
            CollectingPoints(data,num)
            

0
Volume: 584.0180498694999 Count of Contour Data 33
1
Volume: 462.6174827750001 Count of Contour Data 25
2
Volume: 637.2799236839992 Count of Contour Data 42


In [115]:
import pandas as pd
import ipympl as ipv

csvData = pd.read_csv('structure3.csv')
X, Y, Z = csvData.iloc[:,0].values, csvData.iloc[:,1].values, csvData.iloc[:,2].values

In [116]:
# Plot X,Y,Z

fig = plt.figure(figsize= (10, 10))
ax = fig.add_subplot(1,1,1, projection='3d')
#ax.plot_trisurf(X, Y, Z, color='white', edgecolors='red', alpha=0.5)

ax.plot(X, Y, Z, c='red')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …